In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: matplotlib in d:\virtualenv\myenv\lib\site-packages (3.9.0)




[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
from config import Config
import os
import numpy as np
from utils import normalize
from custom import CustomLeakyReLU, CustomClippedReLU

In [ ]:
model_path = './models/dl/soc_estimation_dl.keras'
loaded_model = keras.models.load_model(
    model_path, 
    custom_objects={
        'CustomLeakyReLU': CustomLeakyReLU,
        'CustomClippedReLU': CustomClippedReLU
    }
)

In [ ]:
config = Config(
    data_dir='./data',
    train_data_subdir='train/',
    val_data_subdir='test/',
    test_data_subdir='test/'
)

In [ ]:
test_data_filename = os.listdir(config.get_test_data_dir())[2]
test_data_path = os.path.join(config.get_test_data_dir(), test_data_filename)
test_data = pd.read_csv(test_data_path)
test_data

In [ ]:
X_test = test_data[['V', 'I', 'Temp', 'V_avg', 'I_avg']].values
X_test.shape

In [ ]:
X_test

In [ ]:
normalized_X_test = normalize(X_test)
normalized_X_test

In [ ]:
y_test = test_data['SOC'].values
y_test.shape

In [ ]:
y_test

In [ ]:
soc_observed = y_test

soc_predicted = loaded_model.predict(normalized_X_test)
soc_predicted

In [ ]:
results = loaded_model.evaluate(normalized_X_test, y_test)
results

In [ ]:
from utils import rmse, maxv, mae

rmse_value = rmse(soc_observed, soc_predicted)
print(f'rmse(observed, predicted) = {round(rmse_value*100,2)}%')
max_value = maxv(soc_observed, soc_predicted)
print(f'max(observed, predicted) = {round(max_value*100,2)}%')
mae_value = mae(soc_observed, soc_predicted)
print(f'mae(observed, predicted) = {round(mae_value*100,2)}%')

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(14, 4))

axs[0].plot(soc_observed, label='observed')
axs[0].plot(soc_predicted, label='predicted')
axs[0].set_ylabel('SOC')
axs[0].set_xlabel('Time (s)')
axs[0].legend()

axs[1].plot(soc_observed, label='observed')
axs[1].plot(soc_predicted, label='predicted')
axs[1].set_ylabel('SOC')
axs[1].set_xlabel('Time (s)')
axs[1].legend()
axs[1].axis([0, 10000, 0, 1])

plt.tight_layout()
plt.show()